In [15]:
%pylab inline

from jla_data_fix import *
import pymc3 as pymc
import scipy.integrate as si

Populating the interactive namespace from numpy and matplotlib


`%matplotlib` prevents importing * from pylab and numpy


In [16]:
hubble = 2e-18
sol=3e08

In [17]:
allz = hstack(zs)
allmsb = hstack(msbs)
allmsbe = hstack(msbes)

In [18]:
# Cosmology Functions

def friedmann(z, omegaB):
    model = omegaB * ((1+z)**3) + (1-omegaB)
    
    return sqrt(model)


def integrand(x, b):
    return 1./friedmann(x, b)


def sk(z, omegaB):
    oarray = empty_like(z)
    for i in range(len(oarray)):
        this = si.quad(integrand, 0, z[i], args=(omegaB))
        oarray[i]=this[0]
        
    return oarray


def lumdist(z, omegaB):
    return sk(z, omegaB) * (sol/hubble)


def flux(z, omegaB, lpeak):
    return 1e39*lpeak/(4 * np.pi * (lumdist(z, omegaB)*(1+z))**2)


def msbcalc(z, omegaB, lpeak):
    return -20.45 - 2.5*log10(1e-4 * flux(z, omegaB, lpeak))

vecmsbcalc = vectorize(msbcalc)

In [32]:
import theano as th
import theano.tensor as T

with pymc.Model() as model:
    omegaB = pymc.Uniform('omegaB', lower=0, upper=1)
    leff = pymc.Uniform('leff', lower=0, upper=100)

    y = pymc.Normal('y', msbcalc, sd=(allmsbe)**2, observed=allmsbe)
    
    trace = pm.psample(draws=1000, threads=4)

Applied interval-transform to omegaB and added transformed omegaB_interval to model.
Applied interval-transform to leff and added transformed leff_interval to model.


TypeError: unsupported operand type(s) for -: 'TensorConstant' and 'function'

test with scipy